In [8]:
! pip install python-utils 

In [5]:
from datasets import load_dataset, load_from_disk, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering  ,T5ForConditionalGeneration
from tqdm import tqdm
from IPython.display import clear_output
import torch

from nltk.translate.bleu_score import sentence_bleu

/home/user01/miniconda3/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class AnswerPredictor:
  def __init__(self, model, tokenizer, device='cuda', n_best=10, max_length=512, stride=256, no_answer=False):
      """Initializes PyTorch Question Answering Prediction
      It's best to leave use the default values.
      Args:
          model: Fine-tuned torch model
          tokenizer: Transformers tokenizer
          device (torch.device): Running device
          n_best (int): Number of best possible answers
          max_length (int): Tokenizer max length
          stride (int): Tokenizer stride
          no_answer (bool): If True, model can return "no answer"
      """
      self.model = model.eval().to(device)
      self.tokenizer = tokenizer
      self.device = device
      self.max_length = max_length
      self.stride = stride
      self.no_answer = no_answer
      self.n_best = n_best


  def model_pred(self, questions, contexts, batch_size=1):
      n = len(contexts)
      if n%batch_size!=0:
          raise Exception("batch_size must be divisible by sample length")

      tokens = self.tokenizer(questions, contexts, add_special_tokens=True, 
                              return_token_type_ids=True, return_tensors="pt", padding=True, 
                              return_offsets_mapping=True, truncation="only_second", 
                              max_length=self.max_length, stride=self.stride)

      start_logits, end_logits = [], []
      for i in tqdm(range(0, n-batch_size+1, batch_size)):
          with torch.no_grad():
              out = self.model(tokens['input_ids'][i:i+batch_size].to(self.device), 
                          tokens['attention_mask'][i:i+batch_size].to(self.device), 
                          tokens['token_type_ids'][i:i+batch_size].to(self.device))

              start_logits.append(out.start_logits)
              end_logits.append(out.end_logits)

      return tokens, torch.stack(start_logits).view(n, -1), torch.stack(end_logits).view(n, -1)


  def __call__(self, questions, contexts, batch_size=1, answer_max_len=100):
      """Creates model prediction
      
      Args: 
          questions (list): Question strings
          contexts (list): Contexts strings
          batch_size (int): Batch size
          answer_max_len (int): Sets the longests possible length for any answer
        
      Returns:
          dict: The best prediction of the model
              (e.g {0: {"text": str, "score": int}})
      """
      tokens, starts, ends = self.model_pred(questions, contexts, batch_size=batch_size)
      start_indexes = starts.argsort(dim=-1, descending=True)[:, :self.n_best]
      end_indexes = ends.argsort(dim=-1, descending=True)[:, :self.n_best]

      preds = {}
      for i, (c, q) in enumerate(zip(contexts, questions)):  
          min_null_score = starts[i][0] + ends[i][0] # 0 is CLS Token
          start_context = tokens['input_ids'][i].tolist().index(self.tokenizer.sep_token_id)
          
          offset = tokens['offset_mapping'][i]
          valid_answers = []
          for start_index in start_indexes[i]:
              # Don't consider answers that are in questions
              if start_index<start_context:
                  continue
              for end_index in end_indexes[i]:
                  # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                  # to part of the input_ids that are not in the context.
                  if (start_index >= len(offset) or end_index >= len(offset)
                      or offset[start_index] is None or offset[end_index] is None):
                      continue
                  # Don't consider answers with a length that is either < 0 or > max_answer_length.
                  if end_index < start_index or (end_index-start_index+1) > answer_max_len:
                      continue

                  start_char = offset[start_index][0]
                  end_char = offset[end_index][1]
                  valid_answers.append({"score": (starts[i][start_index] + ends[i][end_index]).item(),
                                        "text": c[start_char: end_char]})
                  
          if len(valid_answers) > 0:
              best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
          else:
              best_answer = {"text": "", "score": min_null_score}

          if self.no_answer:
              preds[i] = best_answer if best_answer["score"] >= min_null_score else {"text": "", "score": min_null_score}
          else:
              preds[i] = best_answer

      return preds

# mBERT

In [8]:
model_path_mBERT = "mbert_checkpoints/checkpoint-6000/" 
model_mBERT = AutoModelForQuestionAnswering.from_pretrained(model_path_mBERT)
tokenizer_mBERT = AutoTokenizer.from_pretrained(model_path_mBERT)
predictor_mBERT = AnswerPredictor(model_mBERT, tokenizer_mBERT, device='cuda', n_best=10, no_answer=True)# mBERT

# ParsBERT

In [9]:

model_path_ParsBERT = "Parsbert" 
model_ParsBERT = AutoModelForQuestionAnswering.from_pretrained(model_path_ParsBERT)
tokenizer_ParsBERT = AutoTokenizer.from_pretrained(model_path_ParsBERT)
predictor_ParsBERT = AnswerPredictor(model_ParsBERT, tokenizer_ParsBERT, device='cuda', n_best=10, no_answer=True)
     


# ParsT5

In [10]:
model_path_ParsT5 = "./model_final" 
device = torch.device('cuda')
model_ParsT5 = T5ForConditionalGeneration.from_pretrained(model_path_ParsT5)
tokenizer_ParsT5 = AutoTokenizer.from_pretrained(model_path_ParsT5)
model_ParsT5.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32103, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32103, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [17]:
class Ensemble:

  def __init__(self,drive_path = './' ,device='cuda', n_best=10, max_length=512, stride=256, no_answer=True):
    
    # ParsBERT
    self.model_path_ParsBERT = 'Parsbert'
    self.model_ParsBERT = AutoModelForQuestionAnswering.from_pretrained(self.model_path_ParsBERT)
    self.tokenizer_ParsBERT = AutoTokenizer.from_pretrained(self.model_path_ParsBERT)
    self.predictor_ParsBERT = AnswerPredictor(self.model_ParsBERT, self.tokenizer_ParsBERT, device='cuda', n_best=10, no_answer=True)     

    # mBERT
    self.model_path_mBERT = "mbert_checkpoints/checkpoint-6000/"
    self.model_mBERT = AutoModelForQuestionAnswering.from_pretrained(self.model_path_mBERT)
    self.tokenizer_mBERT = AutoTokenizer.from_pretrained(self.model_path_mBERT)
    self.predictor_mBERT = AnswerPredictor(self.model_mBERT, self.tokenizer_mBERT, device='cuda', n_best=10, no_answer=True)
          

    #ParsT%
    self.model_path_ParsT5 = "model_final" 
    device = torch.device('cuda')
    self.model_ParsT5 = T5ForConditionalGeneration.from_pretrained(self.model_path_ParsT5)
    self.tokenizer_ParsT5 = AutoTokenizer.from_pretrained(self.model_path_ParsT5)
    self.model_ParsT5.to(device)


  def pred(self, questions, contexts, batch_size=1):

    high_votes = []
    for (question, context) in zip(questions, contexts):

      preds_ParsBERT = self.predictor_ParsBERT([question], [context], batch_size=1)
      preds_mBERT = self.predictor_mBERT([question], [context], batch_size=1)

      input = 'متن: ' + context + '، پرسش: ' + question
      input_ids_ParsT5 = self.tokenizer_ParsT5.encode(input, return_tensors='pt').to(device)
      output_ids_ParsT5 = self.model_ParsT5.generate(input_ids_ParsT5, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
      output_ParsT5 = ' '.join([self.tokenizer_ParsT5.decode(id) for id in output_ids_ParsT5])
      pred_ParsT5 = output_ParsT5.replace('<pad>', '').replace('</s>', '').strip()

      # votes
      preds_scores = [preds_ParsBERT[0], preds_mBERT[0] ]
      preds_text = [preds_ParsBERT[0]["text"], preds_mBERT[0]["text"]  , pred_ParsT5]
      resutls = {}
      for pred in preds_text:
        if pred.strip() not in list(resutls.keys()):
          resutls[pred.strip()] = 1
        else:
          resutls[pred.strip()] += 1

      high_voted = sorted(resutls)[0]
      votes = resutls[high_voted]
      if votes==1 :
        high_voted = sorted(preds_scores , key=lambda x: x["score"], reverse=True)[0]['text']

      high_votes.append(high_voted)
    return high_votes
  

In [18]:
model = Ensemble()

# In one function

In [19]:
contexts = []
questions = []
answers = []

contexts.append('هوا امروز آفتابی است')
questions.append('هوا امروز چگونه است')
answers.append('آفتابی')

contexts.append('خوب، بد، زشت یک فیلم درژانر وسترن اسپاگتی حماسی است که توسط سرجو لئونه در سال ۱۹۶۶ در ایتالیا ساخته شد. زبانی که بازیگران این فیلم به آن تکلم می‌کنند مخلوطی از ایتالیایی و انگلیسی است. این فیلم سومین (و آخرین) فیلم از سه‌گانهٔ دلار (Dollars Trilogy) سرجو لئونه است. این فیلم در حال حاضر در فهرست ۲۵۰ فیلم برتر تاریخ سینما در وب‌گاه IMDB با امتیاز ۸٫۸ از ۱۰، رتبهٔ هشتم را به خود اختصاص داده‌است و به عنوان بهترین فیلم وسترن تاریخ سینمای جهان شناخته می‌شود. «خوب» (کلینت ایستوود، در فیلم، با نام «بلوندی») و «زشت» (ایلای والاک، در فیلم، با نام «توکو») با هم کار می‌کنند و با شگرد خاصی، به گول زدن کلانترهای مناطق مختلف و پول درآوردن از این راه می‌پردازند. «بد» (لی وان کلیف) آدمکشی حرفه‌ای است که به‌خاطر پول حاضر به انجام هر کاری است. «بد»، که در فیلم او را «اِنجل آیز (اِینجل آیز)» (به انگلیسی: Angel Eyes) صدا می‌کنند. به‌دنبال گنجی است که در طی جنگ‌های داخلی آمریکا، به دست سربازی به نام «جکسون»، که بعدها به «کارسون» نامش را تغییر داده، مخفی شده‌است.')
questions.append('در فیلم خوب بد زشت شخصیت ها کجایی صحبت می کنند؟')
answers.append('مخلوطی از ایتالیایی و انگلیسی')

contexts.append('رارداد کرسنت قراردادی برای فروش روزانه معادل ۵۰۰ میلیون فوت مکعب، گاز ترش میدان سلمان است، که در سال ۱۳۸۱ و در زمان وزارت بیژن نامدار زنگنه در دولت هفتم مابین شرکت کرسنت پترولیوم و شرکت ملی نفت ایران منعقد گردید. مذاکرات اولیه این قرارداد از سال ۱۹۹۷ آغاز شد و در نهایت، سال ۲۰۰۱ (۱۳۸۱) به امضای این تفاهم نامه مشترک انجامید. بر اساس مفاد این قرارداد، مقرر شده بود که از سال ۲۰۰۵ با احداث خط لوله در خلیج فارس، گاز فرآورده نشده میدان سلمان (مخزن مشترک با ابوظبی)، به میزان روزانه ۵۰۰ میلیون فوت مکعب (به قول برخی منابع ۶۰۰ میلیون فوت مکعب) به امارات صادر شود. این قرارداد مطابق قوانین داخلی ایران بسته شده‌ و تنها قرارداد نفتی ایران است که از طرف مقابل خود، تضمین گرفته‌است. اجرای این پروژه در سال ۱۳۸۴ با دلایل ارائه شده از سوی دیوان محاسبات ایران از جمله تغییر نیافتن بهای گاز صادراتی و ثابت ماندن آن در هفت سال اول اجرای قرارداد متوقف شد. این در حالی است که طبق تعریف حقوقی، دیوان محاسبات ایران، حق دخالت در قراردادها، پیش از آنکه قراردادها اجرایی و مالی شوند را ندارد.')
questions.append('طرفین قرار داد کرسنت کیا بودن؟	')
answers.append('کرسنت پترولیوم و شرکت ملی نفت ایران')


pred_answers = model.pred(questions,contexts)

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 74.49it/s]


In [20]:
pred_answers

['آفتابی', 'ایتالیایی و انگلیسی', 'شرکت کرسنت پترولیوم و شرکت ملی نفت ایران']

In [22]:
test_dataset = load_from_disk("test.hf").shuffle(seed=42)

In [21]:
from nltk.translate.bleu_score import sentence_bleu


def compute_f1(prediction, answer):
    pred_tokens = prediction.split()
    answer_tokens = answer.split()
    
    if len(pred_tokens) == 0 or len(answer_tokens) == 0:
        return int(pred_tokens == answer_tokens)
    
    common_tokens = set(pred_tokens) & set(answer_tokens)
    
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(answer_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(answer_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_exact_match(prediction, answer):
    return int(prediction == answer)

def bleu(prediction, answer) : 
  reference = [answer.split(' ')]
  candidate = pred.split(' ')
  BLEU = sentence_bleu(reference, candidate)
  BLEU1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
  BLEU4 = sentence_bleu(reference, candidate, weights=(0, 0, 0, 1))

  return BLEU, BLEU1, BLEU4


In [27]:

EH = 0
F1 = 0
BLEU = 0
BLEU1 = 0
BLEU4 = 0
for example in test_dataset: 

  #print(len(example['answers']))
  #if len(example['answers']) != 2: 
    #print(example['answers'])
  if example['answers'] == [] :
    context = example['context'] 
    question = example['question'] 
    preds = model.pred(question, context)[0]
    pred = preds
    if pred == "" : 
      EH += 1
      F1 += 1

    continue

  context = example['context'] 
  question = example['question'] 
  answer = example['answers']
  preds = model.pred(question, context)[0]
  pred = preds

  EH += compute_exact_match(pred, answer)
  F1 += compute_f1(pred, answer)
  b, b1, b4 = bleu(pred, answer)
  BLEU += b
  BLEU1 += b1
  BLEU4 += b4

EH /= len(test_dataset)
F1 /= len(test_dataset)
BLEU /= len(test_dataset)
BLEU1 /= len(test_dataset)
BLEU4 /= len(test_dataset)

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 83.20it/s]


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 82.59it/s]


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 75.91it/s]


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 76.57it/s]


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 79.13it/s]


...

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 82.32it/s]


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 54.19it/s]


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 62.13it/s]


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 65.83it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22   context = example['context']                                                              │
│   23   question = example['question']                                                            │
│   24   answer = example['answers']                                                               │
│ ❱ 25   preds = model.pred(question, context)[0]                                                  │
│   26   pred = preds                                                                              │
│   27                                                                                             │
│   28   EH += compute_exact_match(pred, answer)                                                   │
│                                                                                                  │
│ in pred:36                                                                                       │
│                                                                                                  │
│   33 │                                                                                           │
│   34 │     input = 'متن: ' + context + '، پرسش: ' + question                                     │
│   35 │     input_ids_ParsT5 = self.tokenizer_ParsT5.encode(input, return_tensors='pt').to(dev    │
│ ❱ 36 │     output_ids_ParsT5 = self.model_ParsT5.generate(input_ids_ParsT5, max_length=150, n    │
│   37 │     output_ParsT5 = ' '.join([self.tokenizer_ParsT5.decode(id) for id in output_ids_Pa    │
│   38 │     pred_ParsT5 = output_ParsT5.replace('<pad>', '').replace('</s>', '').strip()          │
│   39                                                                                             │
│                                                                                                  │
│ /home/user01/miniconda3/lib/python3.7/site-packages/torch/autograd/grad_mode.py:27 in            │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/user01/miniconda3/lib/python3.7/site-packages/transformers/generation/utils.py:1534 in     │
│ generate                                                                                         │
│                                                                                                  │
│   1531 │   │   │   │   output_scores=generation_config.output_scores,                            │
│   1532 │   │   │   │   return_dict_in_generate=generation_config.return_dict_in_generate,        │
│   1533 │   │   │   │   synced_gpus=synced_gpus,                                                  │
│ ❱ 1534 │   │   │   │   **model_kwargs,                                                           │
│   1535 │   │   │   )                                       

In [ ]:
EH = 0.67
F1 = 0BLEU = 0
BLEU1 = 0
BLEU4 = 0

In [28]:
print('Exact match of Ensemble on testset : ' + str(EH))
print('F1 score of Ensemble on testset : ' + str(F1))
# print('BLEU score of Ensemble on testset : ' + str(BLEU))
print('BLEU1 score of Ensemble on testset : ' + str(BLEU1))
# print('BLEU4 score of Ensemble on testset : ' + str(BLEU4))
     
